# Customer Churn Prediction with IBM Db2 using Python

__Introduction__

This notebook presents a churn prediction use case using anonymized customer data from a phone operator. It uses IBM Db2 Warehouse and runs on a PySpark kernel. 

Our goal is to accurately predict whether a customer is going to end his/her contract (labeled as positive,1). We prefer to send a commercial email to someone who intends to keep her contract but is labeled as willing to end it (false positive) rather than to overlook the opportunity to prevent a customer from ending her contract (false negative), but also care not to overwhelm our customers with commercials or to lose money by proposing special offers to too many people. Our optimization objective will here be to maximize recall (tp/(tp+fn). We will also look at other classic metrics too.

__Contents__
1. Get ready
2. Explore the data
3. Prepare the data
4. Build your model on the training set
5. Fine tune your hyperparameters on the validation set
6. Assess your model performance on the test set
7. Save your model for future use


__Requirements:__

* PySpark kernel for your Jupyter notebook
* Python 3 including pandas, sklearn, joblib 
* connection to Db2 Warehouse

## 1. Get ready

__Imports__

We import pandas for data exploration and transformation, sklearn for modelisation and evaluation, joblib for model persistence. We will also plot some graphs locally with matplotlib.

In [1]:
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from joblib import dump, load

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
124,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%local
%matplotlib inline

# Useful imports for local visualisations

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import classification_report

__Load the data__

Let's use a table which has been pre-populated in Db2 local. It is called SAMPLES.TRAINING. 

In [3]:
sparkSession = spark \
        .builder \
        .getOrCreate()

df = sparkSession.read \
        .format("com.ibm.idax.spark.idaxsource") \
        .options(dbtable="SAMPLES.TRAINING") \
        .load()
df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----+-----+----------+--------+---------+----------+--------+---------+----------+----------+-----------+------------+---------+----------+-----------+---------+
|CHURN|AREA|VMAIL|VMAIL_MSGS|DAY_MINS|DAY_CALLS|DAY_CHARGE|EVE_MINS|EVE_CALLS|EVE_CHARGE|NIGHT_MINS|NIGHT_CALLS|NIGHT_CHARGE|INTL_MINS|INTL_CALLS|INTL_CHARGE|SVC_CALLS|
+-----+----+-----+----------+--------+---------+----------+--------+---------+----------+----------+-----------+------------+---------+----------+-----------+---------+
|    0| 415|    1|         0|   246.5|      108|     41.91|   216.3|       89|     18.39|     179.6|         99|        8.08|     12.7|         3|       3.43|        2|
|    1| 408|    1|         0|   298.1|      112|     50.68|   201.3|      100|     17.11|     214.7|         88|        9.66|      9.7|         4|       2.62|        2|
|    0| 510|    1|         0|   119.3|       82|     20.28|   185.1|      111|     15.73|     157.0|         74|        7.07|     10.9|         4|       2.

How many records do we have? how many features? 

In [4]:
print('Number of records: '+str(df.count()))
print('Number of features: '+str(len(df.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of records: 3333
Number of features: 17

__Split the data__

We split the data into three datasets: for training, validation and testing. We use the proportions 80%, 10% and 10% so that we keep a relatively high number of examples for training but also have enough examples for the validation and test sets to be representative. Of course these proportions are subjective, you can change them if you want. We have also defined a seed so tht results can be reproduced.

In [5]:
train_DF, val_DF, test_DF = df.randomSplit([0.80,0.10,0.10],1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Convert into Pandas DataFrame__

In the following, we are going to use Python pandas and sklearn rather than Spark SQL and MLlib packages to prepare data and build models.

In [6]:
(train, val, test) = (train_DF.toPandas(), val_DF.toPandas(), test_DF.toPandas())
type(train)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>

## 2. Explore the data

We only explore the training set.

__Data schema__

Let's see what kind of features we have.

In [7]:
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- CHURN: short (nullable = false)
 |-- AREA: integer (nullable = false)
 |-- VMAIL: short (nullable = false)
 |-- VMAIL_MSGS: integer (nullable = false)
 |-- DAY_MINS: decimal(5,1) (nullable = false)
 |-- DAY_CALLS: integer (nullable = false)
 |-- DAY_CHARGE: decimal(5,2) (nullable = false)
 |-- EVE_MINS: decimal(5,1) (nullable = false)
 |-- EVE_CALLS: integer (nullable = false)
 |-- EVE_CHARGE: decimal(5,2) (nullable = false)
 |-- NIGHT_MINS: decimal(5,1) (nullable = false)
 |-- NIGHT_CALLS: integer (nullable = false)
 |-- NIGHT_CHARGE: decimal(5,2) (nullable = false)
 |-- INTL_MINS: decimal(4,1) (nullable = false)
 |-- INTL_CALLS: integer (nullable = false)
 |-- INTL_CHARGE: decimal(4,2) (nullable = false)
 |-- SVC_CALLS: integer (nullable = false)

Columns include:

- Churn : whether the customer resigned his/her phone subscription, this is our target
- Area : a geographic label, we might wonder which impact this feature has on churn
- VMail: whether voice messages have been left
- VMail_msgs: number of voice messages
- day_mins, eve_mins, night_mins, intl_mins: time spend calling in the day/evening/night/abroad
- day_calls, eve_calls, night_calls, intl_calls: number of calls in each category
- day_charge, eve_charge, night_charge, intl_charge: price charged for each category
- svc_calls: service calls.

__Basic statistics__

In [8]:
train_DF.describe().toPandas().transpose()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                  0                    1                    2     3      4
summary       count                 mean               stddev   min    max
CHURN          2639  0.14437286851079956  0.35153402430151326     0      1
AREA           2639   437.88291019325504   42.760111658777824   408    510
VMAIL          2639   0.7154225085259568   0.4512984820521502     0      1
VMAIL_MSGS     2639    8.291777188328913   13.771548440946265     0     51
DAY_MINS       2639            179.37378    53.74770593138116   0.0  350.8
DAY_CALLS      2639   100.39370973853732   19.769098649235545     0    163
DAY_CHARGE     2639            30.494096     9.13705752690767  0.00  59.64
EVE_MINS       2639            202.11849    50.53342222518002   0.0  354.2
EVE_CALLS      2639   100.09814323607426    19.90713448426147     0    170
EVE_CHARGE     2639            17.180288    4.295333768626179  0.00  30.11
NIGHT_MINS     2639            200.49716    50.42337557025347  23.2  395.0
NIGHT_CALLS    2639   100

We see that there is no missing value in the dataset, which is great.

We have a turnout of roughly 15%, so we must do better than a 85% accuracy (accuracy of a naive classifier which would label all examples as 0 id est customer remains a client).


Comments
- most people who have voice mail messages are likely to end their contract, particularly if they receive between 20 and 40 voice messages;
- distribution of number of calls and cumulated time among each category is very similar between classes churn=1 and churn=0. This might make our prediction task hard to perform;
- service calls distributions however differ from class 0 to class 1; 
- area is not a discriminative feauture;
- concentric round patterns tend to appear but the red zone is superimposed with the blue underlying zone, so this might also make prediction hard;
- pricing is linear (simple linear function between time and price, no flat rate or minimum charge).


## 3. Prepare the data

__Add a new column__

Here we create a new column `TOT_MINS` defined as the sum of all call durations. Since charge and duration are bound by a linear function, we will not used `DAY_MINS`, `EVE_MINS`, `INTL_MINS` and `NIGHT_MINS` in the model, just the corresponding `_CHARGE` columns.

In [12]:
def TOT_MINS(dataframe):
    dataframe["TOT_MINS"]=dataframe['DAY_MINS']+dataframe['EVE_MINS']+dataframe['INTL_MINS']+dataframe['NIGHT_MINS']
    return dataframe

train, val, test = TOT_MINS(train), TOT_MINS(val), TOT_MINS(test)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Extract the target vector__

In [13]:
train_y=train["CHURN"]
val_y=val["CHURN"]
test_y=test["CHURN"]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Feature selection with Chi-Test__

In [14]:
# Chi-test for k best features selection

def select_col(X, y, k):
    """
    X: only the features
    y: target vector for X
    k: number of best features to select
    """    
    return SelectKBest(chi2, k=k).fit_transform(X,y)

new_train=select_col(train.drop("CHURN", axis=1), train_y, 6)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# ake a look at the new_train, we have to convert the array back to a pandas dataframe
type(new_train)
new_train=pd.DataFrame(new_train)
new_train.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    0      1      2      3  4      5
0  12  200.3  34.05  253.6  0  613.4
1  13  193.1  32.83  111.6  1  544.2
2  13  207.6  35.29  152.7  1  602.8
3  14   80.2  13.63  219.0  1  430.4
4  15  159.3  27.08  170.6  1  483.0

The column names have been dropped by SelectKBest. We write them here for you and explicitly define the datafames we will use in the next steps for more clarity. The following features have been selected : TOT_MINS, SVC_CALLS, VMAIL_MSGS, DAY_CHARGE, EVE_CHARGE, INTL_CALLS.

In [16]:
train_x=train[["TOT_MINS", "SVC_CALLS", "VMAIL_MSGS", "DAY_CHARGE", "EVE_CHARGE", "INTL_CALLS"]]

val_x=val[["TOT_MINS", "SVC_CALLS", "VMAIL_MSGS", "DAY_CHARGE", "EVE_CHARGE", "INTL_CALLS"]]

test_x=test[["TOT_MINS", "SVC_CALLS", "VMAIL_MSGS", "DAY_CHARGE", "EVE_CHARGE", "INTL_CALLS"]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 4. Train algorithms 

__First try with default options__

Let's define and build a logistic regression model on our training data.

In [17]:
logReg = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1, fit_intercept=True, 
                                        intercept_scaling=1, class_weight=None, random_state=None, 
                                        solver='liblinear', max_iter=100, multi_class='auto', verbose=0, 
                                        warm_start=False, n_jobs=None, l1_ratio=None)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
model = logReg.fit(train_x, train_y)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

__Metrics__

In [19]:
#Accuracy
print("Accuracy on training set")
model.score(train_x, train_y)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy on training set
0.8582796513830997

Just above the accuracy of a naive classifier.

In [20]:
# Confusion matrix
train_pred=model.predict(train_x)
tn, fp, fn, tp = metrics.confusion_matrix(train_y, train_pred).ravel()
print("(tn, fp, fn, tp)="+str((tn, fp, fn, tp)))
recall=tp/(tp+fn)
precision=tp/(tp+fp)
print("Recall: "+str(recall))
print("Precision: "+str(precision))
print("f1-score: "+str(2*(precision*recall)/(precision+recall)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(tn, fp, fn, tp)=(2242, 16, 358, 23)
Recall: 0.06036745406824147
Precision: 0.5897435897435898
f1-score: 0.10952380952380952

## 5. Fine tune your hyperparameters on the validation set

In [21]:
# With our default values : accuracy
model.score(val_x, val_y)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.8304597701149425

In [22]:
# Confusion matrix
val_pred=model.predict(val_x)
tn, fp, fn, tp = metrics.confusion_matrix(val_y, val_pred).ravel()
print(tn, fp, fn, tp)
recall=tp/(tp+fn)
precision=tp/(tp+fp)
print("Recall: "+str(recall))
print("Precision: "+str(precision))
print("f1-score: "+str(2*(precision*recall)/(precision+recall)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

285 4 55 4
Recall: 0.06779661016949153
Precision: 0.5
f1-score: 0.11940298507462686

__Grid Search__

We want to maximize recall id est the proportion of positive cases detected as such over the total number of real positive cases.

In [23]:
def grid_search():
    """
    Input:
    DF : Spark Dataframe, prepared with features and label columns
    
    Output:
    best: dictionary with best hyperparameters and accuracies
    """
    
    # Hyperparameter ranges
    tol_range=[5e-3, 1e-4, 1e-5]
    C_range=[0.02, 0.05, 0.1, 0.5, 1]
    intercept_scaling_range=[50, 20, 10, 2, 1]
    
    best={"tol":0, "C": 0, "intercept_scaling": 0, "training_accuracy": 0, "validation_accuracy": 0, "recall":0}
          
    for tol in tol_range:
        for C in C_range:
            for i in intercept_scaling_range:
            
                print("tol = "+str(tol)+" , C = "+ str(C), "intercept_scaling = "+str(i))

                # Define your model
                logReg = linear_model.LogisticRegression(penalty='l2', dual=False, tol=tol, C=C, fit_intercept=True, 
                                            intercept_scaling=i, class_weight=None, random_state=None, 
                                            solver='liblinear', max_iter=100, multi_class='auto', verbose=0, 
                                            warm_start=False, n_jobs=None, l1_ratio=None)

                # Fit the model on training set
                model = logReg.fit(train_x, train_y)

                # Make predictions on validation set
                val_pred = model.predict(val_x)

                # Compute the accuracy on both datasets
                train_acc = model.score(train_x, train_y)
                val_acc = model.score(val_x, val_y)
                print("Training Accuracy = %g" %  train_acc)
                print("Validation Accuracy = %g" % val_acc)

                # Compute recall
                tn, fp, fn, tp = metrics.confusion_matrix(val_y, val_pred).ravel()
                recall = tp/(tp+fn)
                print("Validation recall: "+str(recall))
                print("")

                # Find the best model
                if recall >= best["recall"]:
                    best["tol"] = tol
                    best["C"] = C
                    best["intercept_scaling"] = i
                    best["training_accuracy"] = train_acc
                    best["validation_accuracy"] = val_acc
                    best["recall"]=recall
                
    return best

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
best=grid_search()
print(best)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

tol = 0.005 , C = 0.02 intercept_scaling = 50
Training Accuracy = 0.861311
Validation Accuracy = 0.836207
Validation recall: 0.03389830508474576

tol = 0.005 , C = 0.02 intercept_scaling = 20
Training Accuracy = 0.861311
Validation Accuracy = 0.833333
Validation recall: 0.01694915254237288

tol = 0.005 , C = 0.02 intercept_scaling = 10
Training Accuracy = 0.856764
Validation Accuracy = 0.83046
Validation recall: 0.0

tol = 0.005 , C = 0.02 intercept_scaling = 2
Training Accuracy = 0.856764
Validation Accuracy = 0.83046
Validation recall: 0.01694915254237288

tol = 0.005 , C = 0.02 intercept_scaling = 1
Training Accuracy = 0.855248
Validation Accuracy = 0.83046
Validation recall: 0.01694915254237288

tol = 0.005 , C = 0.05 intercept_scaling = 50
Training Accuracy = 0.86169
Validation Accuracy = 0.836207
Validation recall: 0.03389830508474576

tol = 0.005 , C = 0.05 intercept_scaling = 20
Training Accuracy = 0.86169
Validation Accuracy = 0.836207
Validation recall: 0.03389830508474576

t

## 6. Assess your model performance on the test set

In [25]:
# Define the model with the hyperparameters you have finally chosen
logReg = linear_model.LogisticRegression(penalty='l2', dual=False, tol=best["tol"], C=best["C"], fit_intercept=True, 
                                            intercept_scaling=best["intercept_scaling"], class_weight=None, random_state=None, 
                                            solver='liblinear', max_iter=100, multi_class='auto', verbose=0, 
                                            warm_start=False, n_jobs=None, l1_ratio=None)
# Fit it to your training set
model=logReg.fit(train_x, train_y)


# Make prediction
test_pred=model.predict(test_x)

# Compute test accuracy
print("Accuracy: "+str(model.score(test_x, test_y)))

# Compute test recall
tn, fp, fn, tp = metrics.confusion_matrix(test_y, test_pred).ravel()
print("(tn, fp, fn, tp)="+str((tn, fp, fn, tp)))
print("Recall: "+str(tp/(tp+fn)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Accuracy: 0.8901734104046243
(tn, fp, fn, tp)=(302, 1, 37, 6)
Recall: 0.13953488372093023

For comparison : with more features ("TOT_MINS", "SVC_CALLS", "VMAIL", "VMAIL_MSGS", "DAY_CHARGE", "EVE_CHARGE", "NIGHT_CHARGE", "INTL_CALLS", "INTL_CHARGE") we obtained:

* 'tol': 1e-05, 
* 'C': 1, 
* 'intercept_scaling': 10, 
* 'training_accuracy': 0.8582796513830997, 
* 'validation_accuracy': 0.8390804597701149, 
* 'recall': 0.1016949152542373

And on the test set:
* Accuracy: 0.884393063583815
* (tn, fp, fn, tp)=(300, 3, 37, 6)
* Recall: 0.13953488372093023

So in our case we have the same recall but a slightly better accuracy.

__Comment on test results__

* 0.89 > 0.85 so we have done better than a naive classifier for accuracy.
* Thanks to hyperparameter tuning, recall was improved by 7% (6.8% -> 14%).
* Recall is still not high but our data didn't allow us to do much better.
* So we have a satisfying result granted that we developed a very basic model.

## 7. Save your model for future use

__Take a look at parameters and hyperparameters__

In [32]:
# hyperparameters of the model
hp=model.get_params()
print(hp)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 2, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'liblinear', 'tol': 1e-05, 'verbose': 0, 'warm_start': False}

In [33]:
# how to transfer hyperparameters to another class
logReg2=linear_model.LogisticRegression().set_params(**hp)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# parameters
print("Parameters for chosen features")
print(model.coef_)
print("Intercept")
print(model.intercept_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Parameters for chosen features
[[ 0.00212649  0.4402351  -0.022415    0.05987647  0.0391706  -0.08033731]]
Intercept
[-5.98505789]

__Save your model with Joblib__

In [ ]:
dump(model, '/tmp/saved_model.joblib') 

In [ ]:
saved_model = load('/tmp/saved_model.joblib') 

In [ ]:
# Test it : you should get the same result as in the previous cell
test_pred2 = saved_model.predict(test_x)
saved_model.score(test_x, test_y)

## What you've learned

Congratulations!

In this notebook you have seen how to:
* fetch sample data from Db2 using PySpark
* use sklearn to build a logistic regression 
* deploy a classic ML workflow from data exploration to model assessment
* save your model for future use - in a Python UDF function for example.

____ 
## Authors

Eva Feillet - ML intern, IBM Cloud and Cognitive Software, Böblingen, Germany

__Licence__
 --> to add